In [0]:
df = spark.table('personalfinance.bronze.gastos_boletos')

In [0]:
from pyspark.sql.functions import when, col, lit, concat, year, month, dayofmonth, lpad

In [0]:
df = df.withColumn('descricao',
                   when(col('file_name').rlike('vivo'), lit('VIVO')) \
                   .when(col('file_name').rlike('ibmec'), lit('IBMEC')) \
                    .when(col('file_name').rlike('canopus'), lit('CANOPUS')) \
                    .otherwise(col('file_name'))                   
                   )

In [0]:
df = df.withColumn('descricao',
                   when(col('valor_brl') > 30_000, lit('CANOPUS ANUAL')) \
                   .otherwise(
                     concat(col('descricao'), 
                            lit(' '), 
                            year(col('data_pagamento')), 
                            lit('_'), 
                            lpad(month(col('data_pagamento')), 2, '0'))))

In [0]:
df = df.withColumn('full_descricao', lit('Boleto')) \
    .withColumn('tipo_compra', lit('Boleto')) \
    .withColumn('bank', lit('Boleto')) \
    .withColumn('card_final', lit('Boleto'))

In [0]:
df = df.withColumnRenamed('data_pagamento', 'data_da_compra') \
    .withColumnRenamed('data_vencimento', 'data_vencimento_fatura') \
    .withColumnRenamed('status', 'status_fatura')

In [0]:
df.write \
    .mode('overwrite') \
    .option('overwriteSchema', 'true') \
    .format('delta') \
    .saveAsTable('personalfinance.silver.gastos_boletos')